In [1]:
from codebase.classes_ibis_lvm import ParticlesLVM
from codebase.classes_data import Data
from codebase.ibis import model_phonebook, essl
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, run_stan_model
from run_ibis_lvm import run_ibis_lvm
from scipy.stats import norm
from tqdm import tqdm
from scipy.special import logsumexp
import altair as alt
from codebase.plot import plot_density, get_post_df, plot_correlations
alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

## 2 factor Sim EZ

In [3]:
log_dir = './log/20210104_220953__/'

## Create Data

In [4]:
data = load_obj('data', log_dir)

## Run IBIS

In [5]:
# load existing results

exp_data = load_obj('data', log_dir)
particles = load_obj('particles', log_dir)

In [6]:
acc_rate = particles.acceptances.astype(float)/particles.counts.astype(float)
acc_rate = pd.DataFrame(np.round(acc_rate[:,particles.ess.astype(int)], 2))
acc_rate['param'] = np.arange(acc_rate.shape[0])
acc_rate = acc_rate.melt(id_vars = 'param', var_name = 't')

c = alt.Chart(acc_rate[acc_rate.param == 2]).mark_bar().encode(
    alt.X('t:Q', title='data iteration t'),
    alt.Y('value:Q')
    ).properties(width=500, height=150)
c


/home/konstantinos/anaconda3/envs/pystan-dev/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.
INFO:numexpr.utils:Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


alt.Chart(...)

## Plot Both results

In [88]:
exp_data.raw_data['beta']
# exp_data.raw_data['N']

array([1. , 0.7, 0.8, 0.5, 0.9, 0.6])

In [89]:
# have to resample particles to get rid of weights
# particles = ibis['particles']
particles.resample_particles()

In [90]:
np.round(np.mean(np.squeeze(particles.particles['beta']),0),2)

array([1.25, 0.38, 0.6 , 0.61, 1.23, 0.56])

In [91]:
param = 'beta'
# df = get_post_df(ps[param].reshape(1000,1,6,1)) 
# df['source'] = 'hmc'
df2 = get_post_df(particles.particles[param])
df2['source'] = 'ibis'
plot_density(df2) 
# &\
# plot_correlations(ibis['jitter_corrs'][param], height = 150)

alt.Chart(...)

In [ ]:
param = 'alpha'
df = get_post_df(ps[param]) 
df['source'] = 'hmc'
df2 = get_post_df(particles.particles[param])
df2['source'] = 'ibis'
plot_density(pd.concat([df,df2])) 